In [ ]:
import pymongo, random, datetime, time, json
from pymongo import MongoClient
import numpy as np
from bson.son import SON
from bson import ObjectId
import pandas as pd

In [ ]:
client = MongoClient(port=27018)
db = client.patterns
current_collection = db.document_versioning_current
historic_collection = db.document_versioning_history
test_collection = db.test

# Setup

In [ ]:
db.drop_collection('document_versioning_current')

In [ ]:
db.drop_collection('document_versioning_history')

In [ ]:
db.drop_collection('test')

In [ ]:
test_collection.insert_one({'event':'video 1'})

# Insert Test

In [ ]:
current_collection.insert_one({'event':'system insert', 
                               'deployment_type':'kubernetes',
                               'created':datetime.datetime.now()})

# Current Document index

In [ ]:
events = list(current_collection.find({}))
print(events[0])

# Historic Document Index

In [ ]:
events = list(historic_collection.find({}))
for event in events:
    print(event)

# Update Test # 1 The update field is present in the schema

In [ ]:
db.drop_collection('document_versioning_current')
db.drop_collection('document_versioning_history')

In [ ]:
current_collection.insert_one({'event':'system insert', 
                                           'deployment_type':'kubernetes',
                                           'created':datetime.datetime.now()
                                          })

In [ ]:
events = list(historic_collection.find({}))
for event in events:
    print(event)

In [ ]:
current_collection.update_one({'_id':events[0]['source_id']}, {'$set':{'event':'system updated'}})

In [ ]:
current_events = list(current_collection.find({}))
for event in current_events:
    print(event)

In [ ]:
historic_events = list(historic_collection.find({}))
for event in historic_events:
    print(event)

# Update Test #2 Update field is not present in the schema

In [263]:
current_collection.update_one({'_id':events[0]['source_id']}, {'$set':{'min_containers':3, 'max_containers':4, 'request.cpu':'1000m'}})

In [264]:
current_events = list(current_collection.find({}))
for event in current_events:
    print(event)

{'_id': ObjectId('623a2164822e42eff6d3943b'), 'event': 'system updated', 'deployment_type': 'kubernetes', 'created': datetime.datetime(2022, 3, 22, 15, 20, 4, 419000), 'max_containers': 4, 'min_containers': 3, 'request': {'cpu': '1000m'}}


In [265]:
historic_events = list(historic_collection.find({}))
for event in historic_events:
    print(event)

{'_id': ObjectId('623a21640339d6672fe18b43'), 'event': 'system updated', 'document_version': 'schema_v1', 'source_id': ObjectId('623a2164822e42eff6d3943b'), 'deployment_type': 'Big Iron', 'created': datetime.datetime(2022, 3, 22, 15, 19, 51, 988000)}
{'_id': ObjectId('623a21640339d6672fe18b44'), 'event': 'system updated', 'document_version': 'schema_v2', 'source_id': ObjectId('623a2164822e42eff6d3943b'), 'deployment_type': 'Cloud VM', 'created': datetime.datetime(2022, 3, 22, 15, 19, 51, 988000)}


# Update Test #3 Some fields are present in the schema but some are not

In [266]:
current_collection.update_one({'_id':events[0]['source_id']}, {'$set':{'event': 'system provisioning started','limit.cpu':'2000m', 'processor_resource_type':'burst', 'tier':2}})

In [267]:
current_events = list(current_collection.find({}))
for event in current_events:
    print(event)

{'_id': ObjectId('623a2164822e42eff6d3943b'), 'event': 'system provisioning started', 'deployment_type': 'kubernetes', 'created': datetime.datetime(2022, 3, 22, 15, 20, 4, 419000), 'max_containers': 4, 'min_containers': 3, 'request': {'cpu': '1000m'}, 'limit': {'cpu': '2000m'}, 'processor_resource_type': 'burst', 'tier': 2}


In [268]:
historic_events = list(historic_collection.find({}))
for event in historic_events:
    print(event)

{'_id': ObjectId('623a21640339d6672fe18b43'), 'event': 'system provisioning started', 'document_version': 'schema_v1', 'source_id': ObjectId('623a2164822e42eff6d3943b'), 'deployment_type': 'Big Iron', 'created': datetime.datetime(2022, 3, 22, 15, 19, 51, 988000)}
{'_id': ObjectId('623a21640339d6672fe18b44'), 'event': 'system provisioning started', 'document_version': 'schema_v2', 'source_id': ObjectId('623a2164822e42eff6d3943b'), 'deployment_type': 'Cloud VM', 'created': datetime.datetime(2022, 3, 22, 15, 19, 51, 988000)}


# Update Test #4 JSON schema validation

In [269]:
big_iron_schema = {
    '$jsonSchema': {
         'required': [ "deployment_type" ],
         'properties': {
             "deployment_type": {
                   'enum': [ "Big Iron" ],
                   'description': "Physical Servers"
                }
         }
    }
}

In [270]:
# searching
list(historic_collection.find(big_iron_schema))

[{'_id': ObjectId('623a21640339d6672fe18b43'),
  'event': 'system provisioning started',
  'document_version': 'schema_v1',
  'source_id': ObjectId('623a2164822e42eff6d3943b'),
  'deployment_type': 'Big Iron',
  'created': datetime.datetime(2022, 3, 22, 15, 19, 51, 988000)}]

In [271]:
historic_collection.update_many(big_iron_schema, {'$set':{'physical_memory_gigabytes':'', 'processor_count':''}})

In [272]:
list(historic_collection.find(big_iron_schema))

[{'_id': ObjectId('623a21640339d6672fe18b43'),
  'event': 'system provisioning started',
  'document_version': 'schema_v1',
  'source_id': ObjectId('623a2164822e42eff6d3943b'),
  'deployment_type': 'Big Iron',
  'created': datetime.datetime(2022, 3, 22, 15, 19, 51, 988000),
  'physical_memory_gigabytes': '',
  'processor_count': ''}]

In [273]:
big_iron_resource_schema = {
    '$jsonSchema': {
         'required': [ "deployment_type", 'physical_memory_gigabytes', 'processor_count' ],
         'properties': {
             "deployment_type": {
                   'enum': [ "Big Iron" ],
                   'description': "Physical Servers"
                },
             "physical_memory_gigabytes":{
                 'bsonType': "int",
                 'minimum': 1,
                 'maximum': 128,
                 'description': "must be an integer in [ 1, 128 ] and is required"
             },
             "processor_count": {
                 'bsonType': "int",
                 'minimum': 1,
                 'maximum': 32,
                 'description': "must be an integer in [ 1, 32 ] and is required"
             },
         }
    }
}

In [274]:
db.command({'collMod':'document_versioning_history', 'validator':big_iron_resource_schema, 'validationLevel':"moderate" })

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1647978869, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}},
 'operationTime': Timestamp(1647978869, 1)}

In [275]:
historic_collection.update_one({'physical_memory_gigabytes':''}, {'$set':{'physical_memory_gigabytes':'129'}})

In [276]:
list(historic_collection.find(big_iron_schema))

[{'_id': ObjectId('623a21640339d6672fe18b43'),
  'event': 'system provisioning started',
  'document_version': 'schema_v1',
  'source_id': ObjectId('623a2164822e42eff6d3943b'),
  'deployment_type': 'Big Iron',
  'created': datetime.datetime(2022, 3, 22, 15, 19, 51, 988000),
  'physical_memory_gigabytes': '129',
  'processor_count': ''}]

# Update Test #5 Document Validation

### Out of Specification

In [277]:
historic_collection.update_one({'physical_memory_gigabytes':'129'}, {'$set':{'physical_memory_gigabytes':129, 'processor_count':4}})

In [278]:
list(historic_collection.find(big_iron_schema))

[{'_id': ObjectId('623a21640339d6672fe18b43'),
  'event': 'system provisioning started',
  'document_version': 'schema_v1',
  'source_id': ObjectId('623a2164822e42eff6d3943b'),
  'deployment_type': 'Big Iron',
  'created': datetime.datetime(2022, 3, 22, 15, 19, 51, 988000),
  'physical_memory_gigabytes': 129,
  'processor_count': 4}]

## In Specification

In [279]:
historic_collection.update_one({'physical_memory_gigabytes':129}, {'$set':{'physical_memory_gigabytes':128, 'processor_count':4}})

In [280]:
list(historic_collection.find(big_iron_schema))

[{'_id': ObjectId('623a21640339d6672fe18b43'),
  'event': 'system provisioning started',
  'document_version': 'schema_v1',
  'source_id': ObjectId('623a2164822e42eff6d3943b'),
  'deployment_type': 'Big Iron',
  'created': datetime.datetime(2022, 3, 22, 15, 19, 51, 988000),
  'physical_memory_gigabytes': 128,
  'processor_count': 4}]

### Out of Specification

In [281]:
historic_collection.update_one({'physical_memory_gigabytes':128}, {'$set':{'physical_memory_gigabytes':256, 'processor_count':64}})

WriteError: Document failed validation, full error: {'index': 0, 'code': 121, 'errInfo': {'failingDocumentId': ObjectId('623a21640339d6672fe18b43'), 'details': {'operatorName': '$jsonSchema', 'schemaRulesNotSatisfied': [{'operatorName': 'properties', 'propertiesNotSatisfied': [{'propertyName': 'physical_memory_gigabytes', 'description': 'must be an integer in [ 1, 128 ] and is required', 'details': [{'operatorName': 'maximum', 'specifiedAs': {'maximum': 128}, 'reason': 'comparison failed', 'consideredValue': 256}]}, {'propertyName': 'processor_count', 'description': 'must be an integer in [ 1, 32 ] and is required', 'details': [{'operatorName': 'maximum', 'specifiedAs': {'maximum': 32}, 'reason': 'comparison failed', 'consideredValue': 64}]}]}]}}, 'errmsg': 'Document failed validation'}